# Charge Time Machine Learning Algorithm Generation

## Table Of Contents

* [1.Reading Data](#read)
  * [1.1 Train-Test Split](#split)
  * [1.2 Data Normalization](#normal)
  * [1.3 Dataframe Variations](#dfs)


* [2.Data Analysis](#split)     


## Reading Data <a name="read"></a>

### Splitting the data <a name="split"></a>

In [1]:
import pandas as pd
import datetime as td
import  numpy as np
import datetime 
import datetime as dt
import math as ma
df = pd.read_csv('data/chargeML.csv')
df=df._convert(numeric=True)
df=df.dropna()
#df['date2'] =  pd.to_datetime(df['date2'])
#df.index.to_pydatetime() 
#df=df.date2.to_pydatetime() 
df['date2']=pd.to_datetime(df.date2)
#df['date2'] =  pd.to_datetime(df['date2'])
df["dayInMonth"] =df['date2'].dt.day
inData=dataSet=df
## Need to add a cos function for day of week

df["cos_week"] = np.cos(df["dayOfWeek"]/7*2*ma.pi)
df["sin_hour"]=np.sin(df["dayOfWeek"]/7*2*ma.pi)

In [2]:
from sklearn.model_selection import train_test_split
import numpy as np

splitOption = 1 # split per day of the month

#cutOffTestDate=2022.01.01
cutOffTestDate = datetime.datetime(2022, 3, 1)
testSet = dataSet.loc[(inData.date2 > cutOffTestDate), :]
mainSet = dataSet.loc[(inData.date2 <= cutOffTestDate), :]



### Normalising the data  <a name="normal"></a>

In [3]:
# Splitting the data by the 23rd of each month, to allow for subsequent hours to be used for predictions.
#Data can also be split by a single cut off date too

# Trainset is the untrasformed data
trainSet = dataSet.loc[(dataSet.dayInMonth > 0) & (dataSet.dayInMonth < 23), :]
validSet = dataSet.loc[(dataSet.dayInMonth >= 23) & (inData.date2 <= cutOffTestDate), :]

# Choosing a predcited column
predCol =['ActualWind']


# Saving these columns in case 
ogWindTrain = trainSet['ActualWind']
ogWindValid = validSet['ActualWind']


#Creating the trainDate column, so it can be re-added later
trainDate=trainSet.date2
validDate=validSet.date2
testDate=testSet.date2

# Removing the date Column
trainSet=trainSet.drop('date2',1)
validSet=validSet.drop('date2',1)
testSet=testSet.drop('date2',1)
trainSet=trainSet.drop('dayInMonth',1)
validSet=validSet.drop('dayInMonth',1)
testSet=testSet.drop('dayInMonth',1)
trainSet=trainSet.drop('dayOfWeek',1)
validSet=validSet.drop('dayOfWeek',1)
testSet=testSet.drop('dayOfWeek',1)
trainSet=trainSet.drop('dayOfYear',1)
validSet=validSet.drop('dayOfYear',1)
testSet=testSet.drop('dayOfYear',1)
trainSet=trainSet.drop('hour',1)
validSet=validSet.drop('hour',1)
testSet=testSet.drop('hour',1)



#When transforming the data in the future, we need to be able to transform the complete forecast,
# for all four stations at once
usedColsForecast=['dub_wddir','dub_wdsp','dub_temp','dub_msl',
    'don_wddir','don_wdsp','don_temp','don_msl',
    'cla_wddir','cla_wdsp','cla_temp','cla_msl',
    'cor_wddir','cor_wdsp','cor_temp','cor_msl']

# Two versions are saved, to allow for the transformed wind-vector columns to be used
usedColsForecastVec=['dub_wddir','dub_wdsp','dub_temp','dub_msl',
    'don_wddir','don_wdsp','don_temp','don_msl',
    'cla_wddir','cla_wdsp','cla_temp','cla_msl',
    'cor_wddir','cor_wdsp','cor_temp','cor_msl',
    'cos_dub_vec', 'sin_dub_vec',
    'cos_cor_vec', 'sin_cor_vec', 'cos_cla_vec', 
    'sin_cla_vec','cos_don_vec', 'sin_don_vec']

# Same as usecColsForecast, but includes time variables
allCols=['sin_day','sin_hour','cos_day','cos_hour',
    'dub_wddir','dub_wdsp','dub_temp','dub_msl',
    'don_wddir','don_wdsp','don_temp','don_msl',
    'cla_wddir','cla_wdsp','cla_temp','cla_msl',
    'cor_wddir','cor_wdsp','cor_temp','cor_msl']

# Same as usecColsForecast, but includes time variables
allCols_Vec=['dub_wddir','dub_wdsp','dub_temp','dub_msl',
    'don_wddir','don_wdsp','don_temp','don_msl',
    'cla_wddir','cla_wdsp','cla_temp','cla_msl',
    'cor_wddir','cor_wdsp','cor_temp','cor_msl',
    'cos_dub_vec', 'sin_dub_vec',
    'cos_cor_vec', 'sin_cor_vec', 'cos_cla_vec', 
    'sin_cla_vec','cos_don_vec', 'sin_don_vec']

usedColsSelect=['sin_day','sin_hour','cos_day','cos_hour',
                'dub_wddir','dub_wdsp','don_wdsp','cor_wdsp',
                'cla_wdsp','dub_msl','don_wdsp','cos_cor_wddir',
                'sin_cor_wddir','dub_temp','cos_dub_wddir','sin_dub_wddir']

usedColsBasic=['sin_day','sin_hour','cos_day','cos_hour',
               'dub_wddir','dub_wdsp','dub_msl']



train_mean = trainSet.mean()
train_std = trainSet.std()


train_meanSave=train_mean[usedColsForecast]
train_stdSave =train_std[usedColsForecast]

# Saving columns which are used to transform data used in future predictive modelling
np.savetxt("./models/train_meanSave.csv", train_meanSave, delimiter=",")
np.savetxt("./models/train_stdSave.csv", train_meanSave, delimiter=",")

train_transformed = (trainSet - train_mean) / train_std
valid_transformed = (validSet - train_mean) / train_std
test_transformed = (testSet - train_mean) / train_std


train_meanSaveVec=train_mean[usedColsForecastVec]
train_stdSaveVec =train_std[usedColsForecastVec]

# Saving columns which are used to transform data used in future predictive modelling
np.savetxt("./models/train_meanSaveVec.csv", train_meanSave, delimiter=",")
np.savetxt("./models/train_stdSaveVec.csv", train_meanSave, delimiter=",")

#re-creating date-column
train_transformed['date2']=trainDate
valid_transformed['date2']=validDate
test_transformed['date2']=testDate

# Re-adding removed columns
train_transformed['hour']=train_transformed['date2'].dt.hour
train_transformed['dayOfWeek']=train_transformed['date2'].dt.day_of_week
train_transformed['dayOfYear']=train_transformed['date2'].dt.day_of_year
train_transformed["dayInMonth"] =train_transformed['date2'].dt.day
# Re-adding removed columns
valid_transformed['hour']=valid_transformed['date2'].dt.hour
valid_transformed['dayOfWeek']=valid_transformed['date2'].dt.day_of_week
valid_transformed['dayOfYear']=valid_transformed['date2'].dt.day_of_year
valid_transformed["dayInMonth"] =valid_transformed['date2'].dt.day
# Re-adding removed columns
test_transformed['hour']=test_transformed['date2'].dt.hour
test_transformed['dayOfWeek']=test_transformed['date2'].dt.day_of_week
test_transformed['dayOfYear']=test_transformed['date2'].dt.day_of_year
test_transformed["dayInMonth"] =test_transformed['date2'].dt.day

#Resetting the predicted  column to not be a transformed variable
train_transformed[predCol]=train_transformed[predCol]*train_std[predCol]+train_mean[predCol]
valid_transformed[predCol]=valid_transformed[predCol]*train_std[predCol]+train_mean[predCol]
test_transformed[predCol]=test_transformed[predCol]*train_std[predCol]+train_mean[predCol]

# Re-setting the names of the columns
train_transformed
valid_transformed
testSet=test_transformed


y_trainWind = trainSet['ActualWind']
y_validWind = validSet['ActualWind']


y_testWind = testSet[predCol]
X_train = trainSet
X_valid = validSet 


/tmp/ipykernel_52086/1205477737.py:23: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  trainSet=trainSet.drop('date2',1)
/tmp/ipykernel_52086/1205477737.py:24: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  validSet=validSet.drop('date2',1)
/tmp/ipykernel_52086/1205477737.py:25: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  testSet=testSet.drop('date2',1)
/tmp/ipykernel_52086/1205477737.py:26: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  trainSet=trainSet.drop('dayInMonth',1)
/tmp/ipykernel_52086/1205477737.py:27: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only

## Dataframe Generation  <a name="dfs"></a>

In [4]:

# Creating dataframes
#Version 1:
X_trainAll = train_transformed[allCols]
X_validAll = valid_transformed[allCols]

X_trainAllVec = train_transformed[allCols_Vec]
X_validAllVec = test_transformed[allCols_Vec]


#Simplified Version:
X_trainBasic = train_transformed[usedColsBasic]
X_validBasic = test_transformed[usedColsBasic]


## Random forest generation

### Random Forest Reggresion model

In [5]:
from sklearn.ensemble import RandomForestRegressor

rrfAll=RandomForestRegressor(n_estimators=500,random_state = 42, max_depth=400, max_leaf_nodes=400)

rrfAll.fit(X_trainAll,y_trainWind)

KeyboardInterrupt: 

In [ ]:
y_predRRFAll=rrfAll.predict(X_validAll)
import matplotlib.pyplot as plt 
from matplotlib.pyplot import figure

plt.plot(X_valid['date2'][-6*24:],y_predRRFAll[-6*24:],color = 'blue',label = 'RFF Predicted Wind Geneartion')
plt.plot(X_valid['date2'][-6*24:],y_valid[-6*24:],color = 'red',label = 'Actual Wind Generation')
plt
fig = plt.gcf()
plt.legend()
#plt.size(7,7)
fig.set_size_inches(11, 7)

fig.savefig('MLImages/rForestForecast2.png', dpi=100)

In [ ]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_valid, y_predRRFAll))
print('Mean Squared Error:', metrics.mean_squared_error(y_valid, y_predRRFAll))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_valid, y_predRRFAll)))

In [ ]:
plt.barh(X_validAll.columns, rrfAll.feature_importances_)
plt.title("Random Forest Example Feature Importantences")
plt.savefig("MLImages/RforestImportances1.png")
plt.show()

In [ ]:
rrfBasic=RandomForestRegressor(n_estimators=200,random_state = 42)
rrfBasic.fit(X_trainBasic,y_trainWind)

In [ ]:
y_predRffBasic=rrfBasic.predict(X_validBasic)

In [ ]:

import matplotlib.pyplot as plt 
from matplotlib.pyplot import figure

plt.plot(X_valid['date2'][-6*24:],y_predRffBasic[-6*24:],color = 'blue')
plt.plot(X_valid['date2'][-6*24:],y_valid[-6*24:],color = 'red')
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
fig.savefig('randomForestVec.png', dpi=100)

In [ ]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_valid,y_predRffBasic))
print('Mean Squared Error:', metrics.mean_squared_error(y_valid, y_predRffBasic))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_valid, y_predRffBasic)))

In [ ]:
plt.barh(X_validBasic.columns, rrf2.feature_importances_)

This:-
rrf=RandomForestRegressor(n_estimators=100,random_state = 42, max_depth=100, max_leaf_nodes=100)

Yields this:-
Mean Absolute Error: 465.79800324008335
Mean Squared Error: 332345.30211887945
Root Mean Squared Error: 576.4939740525302

This:-
rrf=RandomForestRegressor(n_estimators=100,random_state = 42, max_depth=100, max_leaf_nodes=100)

Yields this:-
Mean Absolute Error: 479.4909188704617
Mean Squared Error: 362932.07708576124
Root Mean Squared Error: 602.438442569663

## Creating a Baseline Model Using Tensorflow

In [ ]:
import tensorflow_decision_forests as tfdf

In [ ]:
trainSetTF=trainSet
validSetTF=validSet
testSetTF=testSet

In [ ]:
# Running into difficulty converting these columns, so I reset the value of X_train to include ActualGenerationMW
label = predCol[0]
'''
y_trainTF = tfdf.keras.pd_dataframe_to_tf_dataset(y_trainTF, label=label, task=tfdf.keras.Task.REGRESSION)
y_validTF = tfdf.keras.pd_dataframe_to_tf_dataset(y_validTF, label=label, task=tfdf.keras.Task.REGRESSION)
y_testTF = tfdf.keras.pd_dataframe_to_tf_dataset(y_testTF, label=label, task=tfdf.keras.Task.REGRESSION)
'''

X_trainAllTF = tfdf.keras.pd_dataframe_to_tf_dataset(trainSet[allCols+predCol], label=label, task=tfdf.keras.Task.REGRESSION)

X_validAllTF = tfdf.keras.pd_dataframe_to_tf_dataset(validSetTF[allCols+predCol], label=label, task=tfdf.keras.Task.REGRESSION)

In [ ]:
#Using keras
model_7 = tfdf.keras.RandomForestModel(task = tfdf.keras.Task.REGRESSION)

# Train the model.
model_7.fit(x=X_trainAllTF)

In [ ]:
### Another TF model

using_col_list = ['ForecastWind','sin_day','sin_hour','cos_day','cos_hour','dub_wddir','dub_wdsp','dub_msl','ActualGenerationMW']
using_col_list.remove('ActualGenerationMW')
feature_list = []
for col in using_col_list:
    feature_list.append(tfdf.keras.FeatureUsage(name=col,semantic=tfdf.keras.FeatureSemantic.NUMERICAL))


model8 = tfdf.keras.RandomForestModel(features = feature_list, task = tfdf.keras.Task.REGRESSION,
    exclude_non_specified_features=True,
    num_trees=1000, max_depth=16,
    split_axis="SPARSE_OBLIQUE",categorical_algorithm="RANDOM",
    missing_value_policy='RANDOM_LOCAL_IMPUTATION',
    sparse_oblique_normalization='STANDARD_DEVIATION',
    compute_oob_variable_importances=True,
    winner_take_all=False)
                                                

#model8.compile(metrics=["Accuracy"])


In [ ]:
from wurlitzer import sys_pipes
with sys_pipes():
    model8.fit(X_trainTF)

In [ ]:
# Not sure what test_ds should be
'''
# Evaluate the model on the test dataset.
model8.compile(metrics=["mse"])
evaluation = model8.evaluate(test_ds, return_dict=True)

print(evaluation)
print()
print(f"MSE: {evaluation['mse']}")
print(f"RMSE: {math.sqrt(evaluation['mse'])}")
#print(f"MAE: {evaluation['mae']}")
'''

Not sure if I'll get much better then above

## Tenserflow Setup

In [ ]:
train_df_with_predCol = train_transformed[allCols+predCol]
val_df_with_predCol = valid_transformed[allCols+predCol]
test_df_with_predCol= test_transformed[allCols+predCol]

In [ ]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df=train_df, val_df=val_df, test_df=test_df,
               label_columns=None):
    # Store the raw data.
    self.train_df_with_predCol = train_df_with_predCol
    self.val_df_with_predCol = val_df_with_predCol
    self.test_df_with_predCol= test_df3

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df_with_predCol.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])



In [ ]:
@property
def train(self):
  return self.make_dataset(self.train_df_with_predCol)

@property
def val(self):
  return self.make_dataset(self.val_df_with_predCol)

@property
def test(self):
  return self.make_dataset(self.test_df3)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example



In [ ]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [ ]:
def plot(self, model=None, plot_col='ActualGenerationMW', max_subplots=3):
  inputs, labels = self.example
  plt.figure(figsize=(12, 8))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(max_n, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]')
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
      label_col_index = plot_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot = plot


# Baseline Model

In [ ]:
import tensorflow as tf

In [ ]:
column_indices = {name: i for i, name in enumerate(df.columns)}

n = len(df)
train_df = df[0:int(n*0.7)]
val_df = df[int(n*0.7):int(n*0.9)]
test_df = df[int(n*0.9):]

num_features = df.shape[1]
single_step_window = WindowGenerator(
    input_width=1, label_width=1, shift=1,
    label_columns=['ActualGenerationMW'])
single_step_window

In [ ]:
def make_dataset(self, data):
  data = np.array(data, dtype=np.float32)
  ds = tf.keras.utils.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=1,
      shuffle=True,
      batch_size=32,)

  ds = ds.map(self.split_window)

  return ds

WindowGenerator.make_dataset = make_dataset

In [ ]:
class Baseline(tf.keras.Model):
  def __init__(self, label_index=None):
    super().__init__()
    self.label_index = label_index

  def call(self, inputs):
    if self.label_index is None:
      return inputs
    result = inputs[:, :, self.label_index]
    return result[:, :, tf.newaxis]

baseline = Baseline(label_index=column_indices['ActualGenerationMW'])

baseline.compile(loss=tf.keras.losses.MeanSquaredError(),
                 metrics=[tf.keras.metrics.MeanAbsoluteError()])

val_performance = {}
performance = {}
val_performance['Baseline'] = baseline.evaluate(single_step_window.val)
performance['Baseline'] = baseline.evaluate(single_step_window.test, verbose=0)

In [ ]:
wide_window = WindowGenerator(
    input_width=24, label_width=24, shift=1,
    label_columns=['ActualGenerationMW'])

wide_window

In [ ]:
print('Input shape:', wide_window.example[0].shape)
print('Output shape:', baseline(wide_window.example[0]).shape)
wide_window.plot(baseline)

## Convolutional Neural Network


In [ ]:
'''# I think this could be important,
# If convulutional width is set to 3, then given 3 hours of inputs it predicts 1 hour into the future
# We want for 
CONV_WIDTH = 3

conv_window = WindowGenerator(
    input_width=CONV_WIDTH,
    label_width=1,
    shift=1,
    label_columns=['ActualGenerationMW'])

conv_window'''

In [ ]:
'''conv_model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=32,
                           kernel_size=(CONV_WIDTH,),
                           activation='relu'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=1),
])'''

## Artificial Neural Networks

In [ ]:
import tensorflow as tf
ann = tf.keras.models.Sequential([            
          tf.keras.layers.Dense(units=111, activation='relu', name="Layer_1"),
          tf.keras.layers.Dropout(0.1), #drop-out layer to avoid overfit
          tf.keras.layers.Dense(units=111, activation='relu', name="Layer_2"),
          tf.keras.layers.Dropout(0.1),
          tf.keras.layers.Dense(units=111, activation='relu', name="Layer_3"),
          tf.keras.layers.Dropout(0.1),
          tf.keras.layers.Dense(units=10, activation='relu', name="Layer_4"),
          tf.keras.layers.Dense(units=1, name="output_layer")
          ])

ann.compile(loss=tf.keras.losses.mae, # mae is short for mean absolute error
            optimizer=tf.keras.optimizers.Adam(lr=0.00002),
            metrics=["mae"])

In [ ]:
#copying stuff from document...
epochs=60
learning_rate = 0.1
decay_rate = learning_rate / epochs
momentum = 0.8

#sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
ann.compile(loss=tf.keras.losses.mae, # mae is short for mean absolute error
            optimizer=tf.keras.optimizers.Adam(lr=0.00002),
            metrics=["mae"])

In [ ]:
y_trainWind= trainSet.ActualWind

In [ ]:
ann.fit(X_trainAll,y_trainWind batch_size=20, epochs=20)

In [ ]:
y_predANN=ann.predict(X_validAll)

In [ ]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_valid, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_valid, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_valid, y_pred)))

For lr=0.002 and  batch_size=200, epochs=100)

Mean Absolute Error: 470.42779251935326
Mean Squared Error: 353878.8971401352
Root Mean Squared Error: 594.877211817813

For lr=0.0002 and  batch_size=200, epochs=100)
Mean Absolute Error: 478.03476671201486
Mean Squared Error: 364369.589799041
Root Mean Squared Error: 603.6303420132565

For lr=0.0002 and  batch_size=200, epochs=150)
Mean Absolute Error: 470.0360041216103
Mean Squared Error: 354797.59158343874
Root Mean Squared Error: 595.648882802141

In [ ]:
import matplotlib.pyplot as plt 
from matplotlib.pyplot import figure

plt.plot(X_valid['date2'][-7*24:],y_pred[-7*24:])
plt.plot(X_valid['date2'][-7*24:],y_valid[-7*24:])
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
fig.savefig('test2png.png', dpi=100)

In [ ]:
#plt.barh(X_validAll.columns, ann.feature_importances_)

### Single hour ANN Forecasting

In [ ]:
y_trainSingle.max()

In [ ]:
ann1 = tf.keras.models.Sequential([            
        tf.keras.layers.Dense(units=120, activation='relu', name="Layer_1"),
        tf.keras.layers.Dropout(0.1), #drop-out layer to avoid overfit
        tf.keras.layers.Dense(units=120, activation='relu', name="Layer_2"),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(units=120, activation='relu', name="Layer_3"),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(units=10, activation='relu', name="Layer_4"),
        tf.keras.layers.Dense(units=1, name="output_layer")
        ])

ann1.compile(loss=tf.keras.losses.mae, # mae is short for mean absolute error
          optimizer=tf.keras.optimizers.Adam(lr=0.002),
          metrics=["mae"])



sinTrain1=train_df[allCols]
y_trainSingle=train_df[predCol]

sinValid1=val_df[allCols]
y_validSingle=val_df[predCol]

history = ann1.fit(sinTrain1, y_trainSingle, epochs = 27, validation_data=(sinValid1,y_validSingle), verbose=0)
# history = ann.fit(X_train_normal, y_trainWind epochs = 20, validation_data=(X_valid_normal,y_valid), callbacks=[es])


In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1,len(loss)+1)

plt.plot(epochs,loss,'bo',label='Training loss')
plt.plot(epochs,val_loss, label='Validation loss')
plt.title('Training loss ')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
mask = (weatherMerged['date2'] > '2022-6-21') & (weatherMerged['date2'] <= '2022-6-28')
print(weatherMerged['ActualWind'].loc[mask])

In [ ]:
from sklearn import metrics

y_predS=ann1.predict(sinValid1)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_validSingle, y_predS))
print('Mean Squared Error:', metrics.mean_squared_error(y_validSingle, y_predS))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_validSingle, y_predS)))

In [ ]:
ann1.save('models/ann_modelS')

### Using a different set of columns 

In [ ]:
ann1 = tf.keras.models.Sequential([            
        tf.keras.layers.Dense(units=120, activation='relu', name="Layer_1"),
        tf.keras.layers.Dropout(0.1), #drop-out layer to avoid overfit
        tf.keras.layers.Dense(units=120, activation='relu', name="Layer_2"),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(units=120, activation='relu', name="Layer_3"),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(units=10, activation='relu', name="Layer_4"),
        tf.keras.layers.Dense(units=1, name="output_layer")
        ])

ann1.compile(loss=tf.keras.losses.mae, # mae is short for mean absolute error
          optimizer=tf.keras.optimizers.Adam(lr=0.002),
          metrics=["mae"])





y_trainSingle=train_df[predCol]
y_validSingle=val_df[predCol]

sinTrain1=train_df[allColsForecastVec]
sinValid1=val_df[allColsForecastVec]


history = ann1.fit(sinTrain1, y_trainSingle, epochs = 30, validation_data=(sinValid1,y_validSingle), verbose=0)
# history = ann.fit(X_train_normal, y_trainWind epochs = 20, validation_data=(X_valid_normal,y_valid), callbacks=[es])


In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1,len(loss)+1)

plt.plot(epochs,loss,'bo',label='Training loss')
plt.plot(epochs,val_loss, label='Validation loss')
plt.title('Training loss ')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
sdfdf

In [ ]:
ann.save('models/ann_model')

## Muliple hour input Forecasting

Below from https://www.tensorflow.org/tutorials/structured_data/time_series

In [ ]:
import IPython
import IPython.display

In [ ]:
w2 = WindowGenerator(input_width=6, label_width=1, shift=0,
                     label_columns=['ActualGenerationMW'])
w2

In [ ]:


# Stack three slices, the length of the total window.
example_window = tf.stack([np.array(train_df_with_predCol[:w2.total_window_size]),
                           np.array(train_df_with_predCol[100:100+w2.total_window_size]),
                           np.array(train_df_with_predCol[200:200+w2.total_window_size])])

example_inputs, example_labels = w2.split_window(example_window)

print('All shapes are: (batch, time, features)')
print(f'Window shape: {example_window.shape}')
print(f'Inputs shape: {example_inputs.shape}')
print(f'Labels shape: {example_labels.shape}')

In [ ]:
CONV_WIDTH = 3
conv_window = WindowGenerator(
    input_width=CONV_WIDTH,
    label_width=1,
    shift=0,
    label_columns=['ActualGenerationMW'])

conv_window

In [ ]:
conv_window.plot()
plt.title("Given 3 hours of inputs, predict 1 hour into the future.")

In [ ]:
MAX_EPOCHS = 20

def compile_and_fit(model, window, patience=2):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=patience,mode='min')

  model.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.keras.metrics.MeanAbsoluteError()])

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

### Multi-step dense model

In [ ]:
multi_step_dense = tf.keras.Sequential([
    # Shape: (time, features) => (time*features)
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=1),
    # Add back the time dimension.
    # Shape: (outputs) => (1, outputs)
    tf.keras.layers.Reshape([1, -1]),
])

In [ ]:
history = compile_and_fit(multi_step_dense, conv_window)

IPython.display.clear_output()
val_performance['Multi step dense'] = multi_step_dense.evaluate(conv_window.val)
performance['Multi step dense'] = multi_step_dense.evaluate(conv_window.test, verbose=0)

### conv_model

In [ ]:
conv_model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=32,kernel_size=(CONV_WIDTH,),activation='relu'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=1),
])

In [ ]:
history = compile_and_fit(conv_model, conv_window)

IPython.display.clear_output()
val_performance['Conv'] = conv_model.evaluate(conv_window.val)
performance['Conv'] = conv_model.evaluate(conv_window.test, verbose=0)

In [ ]:
conv_window.train

In [ ]:
MAX_EPOCHS = 20

def compile_and_fit(model, window, patience=2):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=patience,mode='min')

  model.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.keras.metrics.MeanAbsoluteError()])
  print(window.train)
  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

In [ ]:
train_df_with_predCol = train_df[allCols+predCol]
val_df_with_predCol = val_df[allCols+predCol]
test_df_with_predCol= test_df[allCols+predCol]

In [ ]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df=train_df_with_predCol, val_df=val_df_with_predCol, test_df=test_df3,
               label_columns=None):
    # Store the raw data.
    self.train_df_with_predCol = train_df_with_predCol
    self.val_df_with_predCol = val_df_with_predCol
    self.test_df_with_predCol= test_df3

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df_with_predCol.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]
    print("self.input_indices are given by ",self.input_indices)
    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

##----------------------->
CONV_WIDTH = 24
conv_window = WindowGenerator(
    input_width=CONV_WIDTH,
    label_width=0,
    shift=0,
    label_columns=['ActualGenerationMW'])

conv_window

##----------------------->

@property
def train(self):
  return self.make_dataset(self.train_df_with_predCol)


@property
def trainX(self):
  return self.make_dataset(self.train_df_with_predColX)

@property
def trainY(self):
  return self.make_dataset(self.train_df_with_predColY)


@property
def val(self):
  return self.make_dataset(self.val_df_with_predCol)

@property
def valX(self):
  return self.make_dataset(self.val_df_with_predColX)

@property
def valY(self):
  return self.make_dataset(self.val_df_with_predColY)

@property
def test(self):
  return self.make_dataset(self.test_df3)

@property
def testX(self):
  return self.make_dataset(self.test_df3X)

@property
def testY(self):
  return self.make_dataset(self.test_df3Y)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train

WindowGenerator.val = val


WindowGenerator.test = test



WindowGenerator.example = example
###================================>

def split_window(self, features):
    print("self.input_slice is ",self.input_slice)
    print("self.labels_slice is ",self.labels_slice)
    inputs = features[:, self.input_slice, :]
    labels = features[:, self.labels_slice, :]
    print("labels are given by", labels)
    print("inputs are given by", inputs)

    '''if self.label_columns is not None:
        labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)'''

    # Slicing doesn't preserve static shape information, so set the shapes
    # manually. This way the `tf.data.Datasets` are easier to inspect.
    inputs.set_shape([None, self.input_width, None])
    labels.set_shape([None, self.label_width, None])
    print("labels are now given by", labels)
    print("inputs are now given by", inputs)
    return inputs, labels

WindowGenerator.split_window = split_window

###================================>

def make_dataset(self, data):
    data = np.array(data, dtype=np.float32)
    print("Data is  given by",data.shape)
    
    inputData=data[:,:20]
    targets=data[:,20:]
    print("Input data is given by", inputData.shape)
    print("targets are given by",targets.shape)

    ds = tf.keras.utils.timeseries_dataset_from_array(
        inputData,
        targets,
        sequence_length=self.total_window_size,
        sequence_stride=1,
        shuffle=False,
        batch_size=32,)
    print("ds is given by ",ds)
    #ds = ds.map(self.split_window)

    return ds

WindowGenerator.make_dataset = make_dataset

###================================>

MAX_EPOCHS = 20

def compile_and_fit(model, window, patience=2):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=patience,mode='min')

  model.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.keras.metrics.MeanAbsoluteError()])
  print(window.train)
  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

###================================>

conv_model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=32,kernel_size=(CONV_WIDTH-1,),activation='relu'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=1),
])
###================================>

history = compile_and_fit(conv_model, conv_window)

IPython.display.clear_output()
val_performance['Conv'] = conv_model.evaluate(conv_window.val)
performance['Conv'] = conv_model.evaluate(conv_window.test, verbose=0)

#### Generating basic plots

In [ ]:
import matplotlib.pyplot as plt 
from matplotlib.pyplot import figure

y_predRFF=rrf.predict(X_validAll)

y_valid =        val_df[predCol].iloc[2:]
y_forecasted = val_df['ForecastWind'].iloc[2:]


y_predAr=conv_model.predict(conv_window.val)
y_pred=y_predAr[:,0]
y_forecast = validSet.ForecastWind


y_valid=val_df_with_predCol[predCol]
#og here


plt.plot(X_valid['date2'][-6*24:],y_pred[-6*24:],color = 'cyan',label = 'ANN Modeled Wind Generation')
plt.plot(X_valid['date2'][-6*24:],y_valid[-6*24:],color = 'red',label = 'Actual Wind Generation')
plt.plot(X_valid['date2'][-6*24:],y_forecast[-6*24:],color = 'blue', label = 'ESB Forecasted Wind Generation')
plt.plot(X_valid['date2'][-6*24:],y_predRFF[-6*24:],color = 'purple',label = 'RFF Predicted Wind Geneartion')


fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
plt.legend(loc=1)
fig = plt.gcf()
fig.set_size_inches(15, 7)

fig.savefig('MLImages/ModelWeekDisplay.png', dpi=100)

In [ ]:
from sklearn import metrics 
#og stuff
y_valid =        validSet.ForecastWind.iloc[2:]
y_forecasted = validSet.ActualWind.iloc[2:]

MAEAN2=metrics.mean_absolute_error(y_valid, y_pred)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_valid, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_valid, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_valid, y_pred)))


y_valid2 = validSet['ActualWind']
print('\n')
MAEFOR=metrics.mean_absolute_error(y_valid2, y_predRFF)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_valid2, y_predRFF))
print('Mean Squared Error:', metrics.mean_squared_error(y_valid2, y_predRFF))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_valid2, y_predRFF)))

MAERFF=metrics.mean_absolute_error(y_valid, y_forecasted)
print('\n')
print('Mean Absolute Error:', metrics.mean_absolute_error(y_valid, y_forecasted))
print('Mean Squared Error:', metrics.mean_squared_error(y_valid, y_forecasted))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_valid, y_forecasted)))

MAEANN=metrics.mean_absolute_error(y_valid2, y_predANN)
print('\n')
print('Mean Absolute Error:', metrics.mean_absolute_error(y_valid2, y_predANN))
print('Mean Squared Error:', metrics.mean_squared_error(y_valid2, y_predANN))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_valid2, y_predANN)))



In [ ]:
import numpy as np
import matplotlib.pyplot as plt




width = 0.25


plt.bar(x = 1, height = MAEAN2, color = 'b', width = width, edgecolor = 'black',label='MAE Multi hour')
plt.bar(x = 2, height = MAEANN, color = 'r', width = width, edgecolor = 'black',label='MAE single hour ANN')
plt.bar(x = 3, height = MAEFOR, color = 'purple', width = width, edgecolor = 'purple',label='MAE  Random Forest')

plt.bar(x = 4, height = MAERFF, color = 'cyan', width = width, edgecolor = 'purple',label='MAE Forecast Wind')

plt.xlabel("Model")
plt.ylabel("Mean Absolute Error")
plt.title("Accuracy of various ML Algorithms")

# plt.grid(linestyle='--')
#plt.xticks(r + width/2,['2018','2019','2020','2021'])
plt.legend(loc=[0,0])
plt.savefig('MLImages/AccuracyScores.png')
plt.show()


In [ ]:
conv_model.save('models/conv_model')

#### Can I get a Hell yeah
_Helia_

In [ ]:
def plot(self, model=None, plot_col='ActualGenerationMW', max_subplots=3):
  inputs, labels = self.example
  plt.figure(figsize=(12, 8))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(max_n, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]')
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
      label_col_index = plot_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot = plot



LABEL_WIDTH = 24
INPUT_WIDTH = LABEL_WIDTH + (CONV_WIDTH - 1)
wide_conv_window = WindowGenerator(
    input_width=INPUT_WIDTH,
    label_width=LABEL_WIDTH,
    shift=1,
    label_columns=['ActualGenerationMW'])

wide_conv_window

wide_conv_window.plot(conv_model)

In [ ]:
adsf;ligb

In [ ]:
usedCols=['sin_day','sin_hour','cos_day','cos_hour','dub_wddir','dub_wdsp','dub_temp','dub_msl',
    'don_wddir','don_wdsp','don_temp','don_msl',
    'cla_wddir','cla_wdsp','cla_temp','cla_msl',
    'cor_wddir','cor_wdsp','cor_temp','cor_msl','ActualWind']


train_df_with_predCol = train_df[usedCols]
val_df_with_predCol = val_df[usedCols]
test_df_with_predCol= test_df[usedCols]

In [ ]:

class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df=train_df_with_predCol, val_df=val_df_with_predCol, test_df=test_df3,
               label_columns=None):
    # Store the raw data.
    self.train_df_with_predCol = train_df_with_predCol
    self.val_df_with_predCol = val_df_with_predCol
    self.test_df_with_predCol= test_df3

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df_with_predCol.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]
    print("self.input_indices are given by ",self.input_indices)
    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

##----------------------->
CONV_WIDTH = 3
conv_window = WindowGenerator(
    input_width=CONV_WIDTH,
    label_width=0,
    shift=0,
    label_columns=['ActualGenerationMW'])

conv_window

##----------------------->

@property
def train(self):
  return self.make_dataset(self.train_df_with_predCol)


@property
def trainX(self):
  return self.make_dataset(self.train_df_with_predColX)

@property
def trainY(self):
  return self.make_dataset(self.train_df_with_predColY)


@property
def val(self):
  return self.make_dataset(self.val_df_with_predCol)

@property
def valX(self):
  return self.make_dataset(self.val_df_with_predColX)

@property
def valY(self):
  return self.make_dataset(self.val_df_with_predColY)

@property
def test(self):
  return self.make_dataset(self.test_df3)

@property
def testX(self):
  return self.make_dataset(self.test_df3X)

@property
def testY(self):
  return self.make_dataset(self.test_df3Y)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train

WindowGenerator.val = val


WindowGenerator.test = test



WindowGenerator.example = example
###================================>

def split_window(self, features):
    print("self.input_slice is ",self.input_slice)
    print("self.labels_slice is ",self.labels_slice)
    inputs = features[:, self.input_slice, :]
    labels = features[:, self.labels_slice, :]
    print("labels are given by", labels)
    print("inputs are given by", inputs)

    '''if self.label_columns is not None:
        labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)'''

    # Slicing doesn't preserve static shape information, so set the shapes
    # manually. This way the `tf.data.Datasets` are easier to inspect.
    inputs.set_shape([None, self.input_width, None])
    labels.set_shape([None, self.label_width, None])
    print("labels are now given by", labels)
    print("inputs are now given by", inputs)
    return inputs, labels

WindowGenerator.split_window = split_window

###================================>

def make_dataset(self, data):
    data = np.array(data, dtype=np.float32)
    print("Data is  given by",data.shape)
    
    inputData=data[:,:20]
    targets=data[:,20:]
    print("Input data is given by", inputData.shape)
    print("targets are given by",targets.shape)

    ds = tf.keras.utils.timeseries_dataset_from_array(
        inputData,
        targets,
        sequence_length=self.total_window_size,
        sequence_stride=1,
        shuffle=False,
        batch_size=32,)
    print("ds is given by ",ds)
    #ds = ds.map(self.split_window)

    return ds

WindowGenerator.make_dataset = make_dataset

###================================>

MAX_EPOCHS = 20

def compile_and_fit(model, window, patience=2):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=patience,mode='min')

  model.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.keras.metrics.MeanAbsoluteError()])
  print(window.train)
  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

###================================>

conv_model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=32,kernel_size=(CONV_WIDTH-1,),activation='relu'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=1),
])
###================================>

history = compile_and_fit(conv_model, conv_window)

IPython.display.clear_output()
val_performance['Conv'] = conv_model.evaluate(conv_window.val)
performance['Conv'] = conv_model.evaluate(conv_window.test, verbose=0)

In [ ]:
LABEL_WIDTH = 24
INPUT_WIDTH = LABEL_WIDTH + (CONV_WIDTH - 1)
wide_conv_window = WindowGenerator(
    input_width=INPUT_WIDTH,
    label_width=LABEL_WIDTH,
    shift=1,
    label_columns=['ActualGenerationMW'])

wide_conv_window

wide_conv_window.plot(conv_model)

### Transforming new data for the model

### Saving the model in a directory

In [ ]:
conv_model.save('models/conv_model')

## Comparing baselines - Predicting wind power

In [ ]:
train_df

In [ ]:
train_df_with_predCol = train_df[allCols+predCol]
val_df_with_predCol = val_df[allCols+predCol]
test_df_with_predCol= test_df[allCols+predCol]

#Resetting the Actual Generation to be non-continuous
train_df_with_predCol[predCol]=train_df_with_predCol[predCol]*train_std[predCol]+train_mean[predCol]
val_df_with_predCol[predCol]=val_df_with_predCol[predCol]*train_std[predCol]+train_mean[predCol]
test_df3[predCol]=test_df3[predCol]*train_std[predCol]+train_mean[predCol]
#og

In [ ]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df=train_df_with_predCol, val_df=val_df_with_predCol, test_df=test_df3,
               label_columns=None):
    # Store the raw data.
    self.train_df_with_predCol = train_df_with_predCol
    self.val_df_with_predCol = val_df_with_predCol
    self.test_df_with_predCol= test_df3

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df_with_predCol.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]
    print("self.input_indices are given by ",self.input_indices)
    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

##----------------------->
CONV_WIDTH = 3
conv_window = WindowGenerator(
    input_width=CONV_WIDTH,
    label_width=0,
    shift=0,
    label_columns=['ActualGenerationMW'])

conv_window

##----------------------->

@property
def train(self):
  return self.make_dataset(self.train_df_with_predCol)


@property
def trainX(self):
  return self.make_dataset(self.train_df_with_predColX)

@property
def trainY(self):
  return self.make_dataset(self.train_df_with_predColY)


@property
def val(self):
  return self.make_dataset(self.val_df_with_predCol)

@property
def valX(self):
  return self.make_dataset(self.val_df_with_predColX)

@property
def valY(self):
  return self.make_dataset(self.val_df_with_predColY)

@property
def test(self):
  return self.make_dataset(self.test_df3)

@property
def testX(self):
  return self.make_dataset(self.test_df3X)

@property
def testY(self):
  return self.make_dataset(self.test_df3Y)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train

WindowGenerator.val = val


WindowGenerator.test = test



WindowGenerator.example = example
###================================>

def split_window(self, features):
    print("self.input_slice is ",self.input_slice)
    print("self.labels_slice is ",self.labels_slice)
    inputs = features[:, self.input_slice, :]
    labels = features[:, self.labels_slice, :]
    print("labels are given by", labels)
    print("inputs are given by", inputs)

    '''if self.label_columns is not None:
        labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)'''

    # Slicing doesn't preserve static shape information, so set the shapes
    # manually. This way the `tf.data.Datasets` are easier to inspect.
    inputs.set_shape([None, self.input_width, None])
    labels.set_shape([None, self.label_width, None])
    print("labels are now given by", labels)
    print("inputs are now given by", inputs)
    return inputs, labels

WindowGenerator.split_window = split_window

###================================>

def make_dataset(self, data):
    data = np.array(data, dtype=np.float32)
    print("Data is  given by",data.shape)
    
    inputData=data[:,:20]
    targets=data[:,20:]
    print("Input data is given by", inputData.shape)
    print("targets are given by",targets.shape)

    ds = tf.keras.utils.timeseries_dataset_from_array(
        inputData,
        targets,
        sequence_length=self.total_window_size,
        sequence_stride=1,
        shuffle=False,
        batch_size=32,)
    print("ds is given by ",ds)
    #ds = ds.map(self.split_window)

    return ds

WindowGenerator.make_dataset = make_dataset

###================================>

MAX_EPOCHS = 20

def compile_and_fit(model, window, patience=2):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=patience,mode='min')

  model.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.keras.metrics.MeanAbsoluteError()])
  print(window.train)
  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

###================================>

conv_model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=32,kernel_size=(CONV_WIDTH-1,),activation='relu'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=1),
])
###================================>

history = compile_and_fit(conv_model, conv_window)

IPython.display.clear_output()
val_performance['Conv'] = conv_model.evaluate(conv_window.val)
performance['Conv'] = conv_model.evaluate(conv_window.test, verbose=0)

In [ ]:
def plot(self, model=None, plot_col=predCol[0], max_subplots=3):
  inputs, labels = self.example
  plt.figure(figsize=(12, 8))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(max_n, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]')
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
      label_col_index = plot_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot = plot



LABEL_WIDTH = 24
INPUT_WIDTH = LABEL_WIDTH + (CONV_WIDTH - 1)
wide_conv_window = WindowGenerator(
    input_width=INPUT_WIDTH,
    label_width=LABEL_WIDTH,
    shift=1,
    label_columns=['ActualGenerationMW'])

wide_conv_window

wide_conv_window.plot(conv_model)

In [ ]:
val_df_with_predCol

In [ ]:
val_df

In [ ]:
import matplotlib.pyplot as plt 
from matplotlib.pyplot import figure

# Creating predicted column
y_predAr=conv_model.predict(conv_window.val)
y_pred=y_predAr[:,0]



# Setting the values
y_valid =        val_df[predCol].iloc[2:]
y_forecasted = val_df['ForecastWind'].iloc[2:]

val_df_with_predColPlot=val_df_with_predCol.iloc[2:]

print(len(y_forecasted))
print(len(y_valid))
print(len(y_pred))
print(len(val_df[predCol]))
print(len(val_df_with_predColPlot[predCol]))

val_df_with_predColPlot['forecasted']=y_forecasted

val_df_with_predColPlot['predicted']=y_pred

plt.plot(X_valid['date2'][-7*24:],y_pred[-7*24:],color = 'cyan',label = 'ESB Forecasted Wind Generation')
plt.plot(X_valid['date2'][-7*24:],y_valid[-7*24:],color = 'red',label = 'Actual Wind Generation')
plt.plot(X_valid['date2'][-7*24:],y_forecasted[-7*24:],color = 'blue', label = 'Kevins Modeled Wind Generation')

plt.legend(loc=1)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
fig.savefig('test2png.png', dpi=100)

In [ ]:
val_df_with_predCol[['forecasted','ActualWind']].diff().abs().mean()

In [ ]:
val_df_with_predColPlot[['forecasted']]

In [ ]:
np.array(val_df_with_predColPlot[predCol])

In [ ]:
np.mean(abs(np.array(val_df_with_predColPlot[['forecasted']])-np.array(val_df_with_predColPlot[predCol])))

In [ ]:
np.mean(abs(np.array(val_df_with_predColPlot[['predicted']])-np.array(val_df_with_predColPlot[predCol])))

In [ ]:
val_df_with_predCol[['ActualWind','predicted']].diff().abs().mean()